In [2]:
#loading raw data

import pandas as pd

# Load your raw CSV file
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("Divvy_Trips_2020_Q1.csv")

# Preview the dataset
print(df.head())


Saving Divvy_Trips_2020_Q1.csv to Divvy_Trips_2020_Q1.csv
            ride_id rideable_type           started_at             ended_at  \
0  EACB19130B0CDA4A   docked_bike  2020-01-21 20:06:59  2020-01-21 20:14:30   
1  8FED874C809DC021   docked_bike  2020-01-30 14:22:39  2020-01-30 14:26:22   
2  789F3C21E472CA96   docked_bike  2020-01-09 19:29:26  2020-01-09 19:32:17   
3  C9A388DAC6ABF313   docked_bike  2020-01-06 16:17:07  2020-01-06 16:25:56   
4  943BC3CBECCFD662   docked_bike  2020-01-30 08:37:16  2020-01-30 08:42:48   

         start_station_name  start_station_id                end_station_name  \
0  Western Ave & Leland Ave               239           Clark St & Leland Ave   
1   Clark St & Montrose Ave               234  Southport Ave & Irving Park Rd   
2    Broadway & Belmont Ave               296        Wilton Ave & Belmont Ave   
3    Clark St & Randolph St                51        Fairbanks Ct & Grand Ave   
4      Clinton St & Lake St                66           Wells 

In [3]:
print(df.info())
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426887 entries, 0 to 426886
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             426887 non-null  object 
 1   rideable_type       426887 non-null  object 
 2   started_at          426887 non-null  object 
 3   ended_at            426887 non-null  object 
 4   start_station_name  426887 non-null  object 
 5   start_station_id    426887 non-null  int64  
 6   end_station_name    426886 non-null  object 
 7   end_station_id      426886 non-null  float64
 8   start_lat           426887 non-null  float64
 9   start_lng           426887 non-null  float64
 10  end_lat             426886 non-null  float64
 11  end_lng             426886 non-null  float64
 12  member_casual       426887 non-null  object 
dtypes: float64(5), int64(1), object(7)
memory usage: 42.3+ MB
None
ride_id               0
rideable_type         0
started_at            0

In [4]:
# Convert date columns
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

# Calculate ride length in seconds
df['ride_length'] = (df['ended_at'] - df['started_at']).dt.total_seconds()

# Filter out invalid rides
df = df[df['ride_length'] > 0].copy()

# Convert to minutes
df['ride_length_min'] = df['ride_length'] / 60


In [5]:
df['weekday'] = df['started_at'].dt.day_name()
df['month'] = df['started_at'].dt.strftime("%B")


In [7]:
# Filter out outlier rides longer than 1 day
df = df[df['ride_length_min'] < 1440]


In [8]:
# Verify datetime conversion
print("\n🕒 Datetime Types Check:")
print(df[['started_at', 'ended_at']].dtypes)

# Check for negative or zero ride lengths
print("\n❌ Invalid Ride Lengths (<= 0):")
invalid_rides = df[df['ride_length'] <= 0]
print(f"Found {len(invalid_rides)} invalid rides.")

# Check for missing values in key columns
print("\n🔍 Missing Values:")
print(df[['ride_id', 'started_at', 'ended_at', 'member_casual']].isnull().sum())

# Check ride length min column exists and is numeric
if 'ride_length_min' in df.columns and pd.api.types.is_numeric_dtype(df['ride_length_min']):
    print("\n✅ ride_length_min exists and is numeric.")
else:
    print("\n❌ ride_length_min missing or not numeric.")

# Check if weekday and month columns exist
time_cols = ['weekday', 'month']
missing_cols = [col for col in time_cols if col not in df.columns]
if not missing_cols:
    print("\n✅ Time columns (weekday, month) exist.")
else:
    print(f"\n❌ Missing time columns: {missing_cols}")

# Summary of ride length statistics
print("\n📊 Ride Length Summary (in minutes):")
print(df['ride_length_min'].describe())



🕒 Datetime Types Check:
started_at    datetime64[ns]
ended_at      datetime64[ns]
dtype: object

❌ Invalid Ride Lengths (<= 0):
Found 0 invalid rides.

🔍 Missing Values:
ride_id          0
started_at       0
ended_at         0
member_casual    0
dtype: int64

✅ ride_length_min exists and is numeric.

✅ Time columns (weekday, month) exist.

📊 Ride Length Summary (in minutes):
count    426387.000000
mean         14.376844
std          34.040008
min           0.016667
25%           5.483333
50%           9.166667
75%          15.800000
max        1435.916667
Name: ride_length_min, dtype: float64


In [9]:
#group by rider type
avg_duration = df.groupby("member_casual")["ride_length_min"].mean().round(2).reset_index()
print(avg_duration)



  member_casual  ride_length_min
0        casual            37.03
1        member            11.50


In [15]:
# group and count rides by weekday and rider type
weekday_usage = df.groupby(['weekday', 'member_casual'])['ride_id'].count().reset_index()
weekday_usage.rename(columns={'ride_id': 'ride_count'}, inplace=True)
print(weekday_usage)




      weekday member_casual  ride_count
0      Friday        casual        5103
1      Friday        member       55477
2      Monday        casual        4785
3      Monday        member       61916
4    Saturday        casual        7445
5    Saturday        member       30095
6      Sunday        casual       14820
7      Sunday        member       35959
8    Thursday        casual        4819
9    Thursday        member       61236
10    Tuesday        casual        5225
11    Tuesday        member       69692
12  Wednesday        casual        5846
13  Wednesday        member       63969


In [11]:
#group by bike type

bike_type_usage = df.groupby(["rideable_type", "member_casual"])["ride_id"].count().reset_index()
print(bike_type_usage)


  rideable_type member_casual  ride_id
0   docked_bike        casual    48043
1   docked_bike        member   378344


In [12]:
#group by month to identify seasonal trends

monthly_usage = df.groupby(["month", "member_casual"])["ride_id"].count().reset_index()
print(monthly_usage)


      month member_casual  ride_id
0  February        casual    12797
1  February        member   126696
2   January        casual     7721
3   January        member   136082
4     March        casual    27525
5     March        member   115566


In [16]:
#summary table files

avg_duration.to_csv("avg_ride_duration.csv", index=False)
weekday_usage.to_csv("weekday_usage.csv", index=False)
bike_type_usage.to_csv("bike_type_usage.csv", index=False)
monthly_usage.to_csv("monthly_usage.csv", index=False)

from google.colab import files

# Download each file
files.download("avg_ride_duration.csv")
files.download("weekday_usage.csv")
files.download("bike_type_usage.csv")
files.download("monthly_usage.csv")




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>